# Importação de base de dados

In [57]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

In [63]:
TRN_X = pd.read_csv('TRN_X.csv')
TRN_Y = pd.read_csv('TRN_Y.csv')

TST_X = pd.read_csv('TST_X.csv')
TST_Y = pd.read_csv('TST_Y.csv')

In [64]:
TRN_X.fillna(0, inplace=True)
TRN_Y.fillna(0, inplace=True)

In [ ]:
TRN_X.head()

In [74]:
TRN_Y.head()
# append value 1 to the end of TRN_Y
TRN_Y['target'] = 1

# Classificador Random Forest

In [75]:
RFC = RandomForestClassifier(n_estimators=100, max_features=30)

RFC.fit(TRN_X, TRN_Y)


ValueError: Found input variables with inconsistent numbers of samples: [50000, 49999]